<a href="https://colab.research.google.com/github/GrzegorzSzczepanek/deep_learning_fun/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My attempt of creating a MLP neural network for predicting the flag.

In [127]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report, roc_curve, roc_auc_score
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn

###Let's define constants for this code

In [128]:
device = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32
EPOCHS = 20

In [129]:
dataset = pd.read_csv("/content/drive/MyDrive/neuron_workplace/autentykacja_eeg.csv", delimiter=";")
dataset.head()

,Delta,Theta,Alpha_low,Alpha_high,Beta_low,Beta_high,Gamma_low,Gamma_high,Attention,Meditation,BlinkStrength,Flag
0,785389,114375,57106,37625,24464,17104,15846,8024,45,91,15,1
1,386949,80792,23506,22118,9566,7147,5392,1697,8,42,6,0
2,21453,14085,4756,5062,2176,2117,2867,862,48,66,9,1
3,322637,56936,17810,25793,15870,8050,3621,2100,25,69,8,0
4,19412,12985,6416,6131,7126,7419,2677,1884,80,53,-1,1


In [130]:
# Try creating validation sets next time
X = torch.tensor(dataset.loc[:, "Delta": "BlinkStrength"].values, dtype=torch.float32)
y = torch.tensor(dataset.Flag.values, dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [131]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([4195, 11]),
 torch.Size([1049, 11]),
 torch.Size([4195]),
 torch.Size([1049]))

## Let's create a datasets and proceed so we can make a dataloaders out of them.

In [132]:
y_train = y_train.unsqueeze(1)
y_test = y_test.unsqueeze(1)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True

)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)
print(f"Train DataLoader length: {len(train_dataloader)} | Batch Size: {BATCH_SIZE}")
print(f"Test DataLoader length: {len(test_dataloader)} | Batch Size: {BATCH_SIZE}")

Train DataLoader length: 132 | Batch Size: 32
Test DataLoader length: 33 | Batch Size: 32


## First of all, I'll try to train my neural network on unchanged data.

In [133]:
input = len(dataset.columns.values) - 1
output = 1


class FlagModel(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()

        self.block_1 = nn.Sequential(
            nn.Linear(in_features=input_shape, out_features=hidden_units),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.Sigmoid(),
        )
        self.block_2 = nn.Sequential(
            nn.Linear(in_features=hidden_units, out_features=output_shape)
        )


    def forward(self, x):
        return self.block_2(self.block_1(x))


flag_model_1 = FlagModel(input, 10, output).to(device)
flag_model_1

FlagModel(
  (block_1): Sequential(
    (0): Linear(in_features=11, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Sigmoid()
  )
  (block_2): Sequential(
    (0): Linear(in_features=10, out_features=1, bias=True)
  )
)

In [134]:
def accuracy_fn(y_true: torch.Tensor, y_pred: torch.Tensor) -> [torch.float]:
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

loss_fn = nn.BCEWithLogitsLoss()
# loss_fn = nn.BCELoss()

optimizer = torch.optim.Adam(params=flag_model_1.parameters(), lr=0.003)

In [135]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    model.train()

    for batch, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)
        logits = model(X)
        pred = torch.round(torch.sigmoid(logits))

        loss = loss_fn(logits, y)
        acc = accuracy_fn(y_true=y, y_pred=pred)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss
        train_acc += acc

    return train_loss / len(data_loader), train_acc / len(data_loader)



In [136]:
def test_step(model: torch.nn.Module,
               data_loader: torch.tensor,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device:torch.device = device):
    model.eval()
    test_loss, test_acc = 0, 0
    with torch.inference_mode():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            logits = model(X)
            pred = torch.round(torch.sigmoid(logits))
            loss = loss_fn(logits, y)
            acc = accuracy_fn(y_true=y, y_pred=pred)

            test_loss += loss
            test_acc += acc

        return loss, acc



In [137]:
from timeit import default_timer as timer
from tqdm.auto import tqdm

train_time_start = timer()

torch.manual_seed(42)

def train(model: torch.nn.Module,
          train_data: torch.utils.data.DataLoader,
          test_data: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          accuracy_fn,
          epochs: int,
          device:torch.device = device):

    for epoch in tqdm(range(EPOCHS)):
        train_loss, train_acc = train_step(model=model,
                                        data_loader=train_dataloader,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer,
                                        accuracy_fn=accuracy_fn)
        test_loss, test_acc = test_step(model=model,
                                        data_loader=test_dataloader,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer,
                                        accuracy_fn=accuracy_fn)


        print(f"Epoch: {epoch} | Train Loss: {train_loss} | Train Acc: {train_acc:.2f}% | Test loss: {test_loss} | Test Acc: {test_acc:.2f}%")

    train_time_end = timer()
    print(f"Training was done in {(train_time_end - train_time_start):.2f} seconds")

In [138]:
train(model=flag_model_1,
      train_data=X_train,
      test_data=X_test,
      loss_fn=loss_fn,
      optimizer=optimizer,
      accuracy_fn=accuracy_fn,
      epochs=EPOCHS)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 0 | Train Loss: 0.6756830215454102 | Train Acc: 59.49% | Test loss: 0.7124903202056885 | Test Acc: 48.00%
Epoch: 1 | Train Loss: 0.6756928563117981 | Train Acc: 59.49% | Test loss: 0.7366316914558411 | Test Acc: 48.00%
Epoch: 2 | Train Loss: 0.6763671040534973 | Train Acc: 59.26% | Test loss: 0.7172093987464905 | Test Acc: 48.00%
Epoch: 3 | Train Loss: 0.6755439639091492 | Train Acc: 59.49% | Test loss: 0.7237324714660645 | Test Acc: 48.00%
Epoch: 4 | Train Loss: 0.6756774187088013 | Train Acc: 59.49% | Test loss: 0.7200071811676025 | Test Acc: 48.00%
Epoch: 5 | Train Loss: 0.6754586696624756 | Train Acc: 59.49% | Test loss: 0.7086459398269653 | Test Acc: 48.00%
Epoch: 6 | Train Loss: 0.6760831475257874 | Train Acc: 59.49% | Test loss: 0.7169122099876404 | Test Acc: 48.00%
Epoch: 7 | Train Loss: 0.6753618717193604 | Train Acc: 59.49% | Test loss: 0.7341742515563965 | Test Acc: 48.00%
Epoch: 8 | Train Loss: 0.6745291948318481 | Train Acc: 59.71% | Test loss: 0.7136450409889221 | 